In [75]:
import pandas as pd
import numpy as np
from numpy.core.records import fromarrays
from scipy.io import savemat

In [76]:
choice = 0  #start from 0
devices = ['nfet_01v8_lvt', 'nfet_01v8', 'pfet_01v8_lvt', 'pfet_01v8']

# widths used for characterization and fringe cap parameters (fringe caps are not included in ngspice output)
w = np.array([5, 5, 5, 5])
nfing = np.array([1, 1, 1, 1])
cgdo_w = np.array([2.39289e-10, 2.44907e-10, 2e-11, 5.24893e-11])*1e-6

In [77]:
df_raw = pd.read_csv('./simulation/techsweep_'+devices[choice]+'.txt', sep='\s+')
par_names = df_raw.columns.to_list()
par_prefix = par_names[1].split('[')[0]
print(par_prefix)

@m.xm1.msky130_fd_pr__nfet_01v8_lvt


In [78]:
# remove extra headers in file body and unwanted columns
df_raw = df_raw[~df_raw['v-sweep'].str.contains('v-sweep')]
df = df_raw.drop(['v-sweep', 'v-sweep.1'], axis=1)
df = df.apply(pd.to_numeric)

# rename columns for readability
df.columns = df.columns.str.removeprefix(par_prefix+'[')
df.columns = df.columns.str.removesuffix(']')
df

,capbd,capbs,cdd,cgb,cgd,cgg,cgs,css,gds,gm,gmbs,id,l,vth,b,d,g
0,2.820000e-15,2.820000e-15,4.190000e-16,-1.460000e-15,-3.334000e-16,1.619000e-15,1.744000e-16,-8.537000e-17,6.077000e-11,0.000000,0.000000,0.000000,1.500000e-07,0.6197,0.0,0.0,0.000
1,2.820000e-15,2.820000e-15,4.794000e-16,-1.436000e-15,-3.814000e-16,1.621000e-15,1.957000e-16,-9.529000e-17,1.156000e-10,0.000000,0.000000,0.000000,1.500000e-07,0.6197,0.0,0.0,0.025
2,2.820000e-15,2.820000e-15,5.463000e-16,-1.411000e-15,-4.344000e-16,1.628000e-15,2.167000e-16,-1.047000e-16,2.200000e-10,0.000000,0.000000,0.000000,1.500000e-07,0.6197,0.0,0.0,0.050
3,2.820000e-15,2.820000e-15,6.198000e-16,-1.386000e-15,-4.927000e-16,1.641000e-15,2.377000e-16,-1.136000e-16,4.186000e-10,0.000000,0.000000,0.000000,1.500000e-07,0.6197,0.0,0.0,0.075
4,2.820000e-15,2.820000e-15,6.997000e-16,-1.361000e-15,-5.559000e-16,1.659000e-15,2.578000e-16,-1.215000e-16,7.963000e-10,0.000000,0.000000,0.000000,1.500000e-07,0.6197,0.0,0.0,0.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255834,1.471000e-15,2.284000e-15,6.614000e-17,1.613000e-15,-5.794000e-17,8.809000e-14,-8.964000e-14,5.367000e-14,4.189000e-06,0.000412,0.000070,0.000276,3.000000e-06,0.5237,-0.4,1.8,1.700
255835,1.471000e-15,2.284000e-15,7.085000e-17,1.749000e-15,-6.208000e-17,8.817000e-14,-8.985000e-14,5.372000e-14,4.362000e-06,0.000417,0.000072,0.000287,3.000000e-06,0.5237,-0.4,1.8,1.725
255836,1.471000e-15,2.284000e-15,7.593000e-17,1.884000e-15,-6.654000e-17,8.824000e-14,-9.006000e-14,5.377000e-14,4.552000e-06,0.000422,0.000074,0.000297,3.000000e-06,0.5237,-0.4,1.8,1.750
255837,1.471000e-15,2.284000e-15,8.141000e-17,2.019000e-15,-7.135000e-17,8.832000e-14,-9.026000e-14,5.381000e-14,4.760000e-06,0.000426,0.000076,0.000308,3.000000e-06,0.5237,-0.4,1.8,1.775


In [79]:
# sweep variable vectors
l = (np.unique(abs(df['l']))/1e-6).round(3)
vgs = np.unique(abs(df['g']))
vds = np.unique(abs(df['d']))
vsb = np.unique(abs(df['b']))

In [80]:
# data
# ngspice sweep order is vgs, vds, vsb, l
dims = [len(vgs), len(vds), len(vsb), len(l)]
id = np.reshape(df['id'].values, dims, order='F')
vt = np.reshape(df['vth'].values, dims, order='F')
gm = np.reshape(df['gm'].values, dims, order='F')
gmb = np.reshape(df['gmbs'].values, dims, order='F')
gds = np.reshape(df['gds'].values, dims, order='F')
cfringe = w[choice]*cgdo_w[choice]
cgg = np.reshape(df['cgg'].values, dims, order='F') \
      + 2*cfringe
cgb = -np.reshape(df['cgb'].values, dims, order='F')
cgd = -np.reshape(df['cgd'].values, dims, order='F') \
      + cfringe
cgs = -np.reshape(df['cgs'].values, dims, order='F') \
      + cfringe
cdd = np.reshape(df['cdd'].values, dims, order='F') \
      + np.reshape(df['capbd'].values, dims, order='F') \
      + cfringe
css = np.reshape(df['css'].values, dims, order='F') \
      + np.reshape(df['capbd'].values, dims, order='F') \
      + cfringe

# output order for standard mat file is l, vgs, vds, vsb
id = np.transpose(id, (3, 0, 1, 2))
vt = np.transpose(vt, (3, 0, 1, 2))
gm = np.transpose(gm, (3, 0, 1, 2))
gmb = np.transpose(gmb, (3, 0, 1, 2))
gds = np.transpose(gds, (3, 0, 1, 2))
cgg = np.transpose(cgg, (3, 0, 1, 2))
cgb = np.transpose(cgb, (3, 0, 1, 2))
cgd = np.transpose(cgd, (3, 0, 1, 2))
cgs = np.transpose(cgs, (3, 0, 1, 2))
cdd = np.transpose(cdd, (3, 0, 1, 2))
css = np.transpose(css, (3, 0, 1, 2))

In [81]:
dic = {
  "INFO": "SkyWater, 130nm CMOS, BSIM4",
  "CORNER": "NOM",
  "TEMP": 300.0,
  "VGS": vgs,
  "VDS": vds,
  "VSB": vsb,
  "L": l,
  "W": w[choice],
  "NFING": nfing[choice],
  "ID": id,
  "VT": vt,
  "GM": gm,
  "GMB": gmb,
  "GDS": gds,
  "CGG": cgg,
  "CGB": cgb,
  "CGD": cgd,
  "CGS": cgs,
  "CDD": cdd,
  "CSS": css
}

savemat('./simulation/'+devices[choice]+'.mat', {devices[choice]: dic})